# Conexión a PostgreSQL con psycopg2: teoría y ejemplos

Este notebook resume los conceptos clave para conectar Python con PostgreSQL usando `psycopg2`, e incluye ejemplos prácticos de conexión, creación de tablas, inserciones seguras con parámetros, lecturas, transacciones (commit/rollback), autocommit y manejo de errores.

## Requisitos y preparación

- Python 3.8+
- PostgreSQL instalado y accesible
- Una base de datos (por ejemplo, `biblioteca`)
- Librería `psycopg2` (para desarrollo local en Windows es práctico `psycopg2-binary`)

Puedes instalar dependencias con:
```bash
pip install -r requirements.txt
```

En este repositorio ya tienes el script `connection_postgres.py` con funciones reutilizables.

## Conceptos clave

- Conexión: `psycopg2.connect(dbname, user, password, host, port)` devuelve un objeto `connection`.
- Cursor: `conn.cursor()` crea un cursor para ejecutar sentencias SQL. Úsalo con `with` para cerrarlo automáticamente.
- Parámetros seguros: usa `%s` y una tupla de valores para evitar inyección SQL.
- Transacciones: por defecto autocommit está desactivado. Usa `conn.commit()` para confirmar o `conn.rollback()` para revertir.
- Manejo de errores: captura `psycopg2.OperationalError` y `psycopg2.Error` para mensajes claros.
- Cierre: cierra cursor y conexión (context managers y bloque `finally`).

## Ejemplo 1: Conexión básica y cierre en `finally`

In [ ]:
# Ejemplo educativo: conexión básica
import psycopg2
from psycopg2 import OperationalError, Error

DB_NAME = 'biblioteca'      # Cambia según tu entorno
DB_USER = 'postgres'        # Cambia según tu entorno
DB_PASSWORD = 'tu_password' # Cambia según tu entorno
DB_HOST = 'localhost'
DB_PORT = 5432

conn = None
try:
    conn = psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
    )
    print('Conexión OK')
    # Aquí podrías ejecutar sentencias usando un cursor
except OperationalError as oe:
    print('[ERROR] Problema operacional al conectar:', oe)
except Error as e:
    print('[ERROR] psycopg2 Error al conectar:', e)
except Exception as ex:
    print('[ERROR] Inesperado:', ex)
finally:
    if conn is not None:
        try:
            conn.close()
            print('Conexión cerrada')
        except Exception as ex_close:
            print('[WARN] No se pudo cerrar limpiamente:', ex_close)

## Ejemplo 2: Usar las funciones del script `connection_postgres.py`

Estas funciones encapsulan buenas prácticas: parámetros seguros, commit/rollback y cierre de cursores.

In [ ]:
# Reutilizar las funciones ya creadas en el repo
from connection_postgres import conectar, crear_tabla, insertar_libro, mostrar_libros
import os
from dotenv import load_dotenv

#Cargar variables desde archivo
load_dotenv()

# Puedes usar variables de entorno para las credenciales
DB_NAME = os.getenv('PG_DB', 'biblioteca')
DB_USER = os.getenv('PG_USER', 'postgres')
DB_PASSWORD = os.getenv('PG_PASSWORD', 'tu_password')
DB_HOST = os.getenv('PG_HOST', 'localhost')
DB_PORT = int(os.getenv('PG_PORT', '5432'))

conn = conectar(DB_NAME, DB_USER, DB_PASSWORD, DB_HOST, DB_PORT)
if conn:
    try:
        crear_tabla(conn)
        insertar_libro(conn, 'El nombre de la rosa', 'Umberto Eco', 1980)
        mostrar_libros(conn)
    finally:
        # Cerrar la conexión cuando termines
        if not getattr(conn, 'closed', 1):
            conn.close()
            print('Conexión cerrada (demo funciones).')

## Ejemplo 3: Parámetros seguros y prevención de inyección SQL

Evita concatenar strings con valores del usuario. Usa `%s` y una tupla de parámetros.

In [ ]:
# MAL: susceptible a inyección (no ejecutar)
# titulo = input('Título: ')
# autor = input('Autor: ')
# anio = int(input('Año: '))
# sql_mal = f"INSERT INTO libros (titulo, autor, anio) VALUES ('{titulo}', '{autor}', {anio});"
# cur.execute(sql_mal)

# BIEN: uso de parámetros
import psycopg2
from psycopg2 import Error

def insertar_seguro(conn, titulo, autor, anio):
    sql = 'INSERT INTO libros (titulo, autor, anio) VALUES (%s, %s, %s);'
    try:
        with conn.cursor() as cur:
            cur.execute(sql, (titulo, autor, anio))
        conn.commit()
        print('[OK] Inserción segura registrada')
    except Error as e:
        conn.rollback()
        print('[ERROR] Insertar seguro falló:', e)

## Ejemplo 4: Transacciones con commit() y rollback()

Controla explícitamente la transacción cuando autocommit está desactivado (comportamiento por defecto).

In [ ]:
import psycopg2
from psycopg2 import Error

def transferencia_ejemplo(conn):
    try:
        with conn.cursor() as cur:
            # Operación 1
            cur.execute("UPDATE cuentas SET saldo = saldo - %s WHERE id = %s;", (100, 1))
            # Operación 2
            cur.execute("UPDATE cuentas SET saldo = saldo + %s WHERE id = %s;", (100, 2))
        conn.commit()
        print('[OK] Transacción confirmada')
    except Error as e:
        conn.rollback()
        print('[ERROR] Transacción revertida:', e)

## Ejemplo 5: Context managers para cursores y buenas prácticas

Usa `with conn.cursor() as cur:` para asegurar cierre del cursor, incluso si hay excepciones.

In [ ]:
def listar_libros(conn):
    sql = 'SELECT id, titulo, autor, anio FROM libros ORDER BY id;'
    try:
        with conn.cursor() as cur:
            cur.execute(sql)
            filas = cur.fetchall()
        for (id_libro, titulo, autor, anio) in filas:
            print(f'#{id_libro:>3} | {titulo:<30} | {autor:<25} | {anio}')
    except Exception as e:
        print('[ERROR] listar_libros:', e)

## Ejemplo 6: Autocommit

`conn.autocommit = True` ejecuta cada sentencia en su propia transacción. Útil para DDL (CREATE/DROP) y comandos administrativos, pero úsalo con cuidado.

In [ ]:
def crear_tabla_autocommit(conn):
    ddl = '''
    CREATE TABLE IF NOT EXISTS libros_autocommit (
        id SERIAL PRIMARY KEY,
        titulo VARCHAR(100)
    );
    '''
    prev = conn.autocommit
    try:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(ddl)
        print('[OK] DDL ejecutado con autocommit')
    finally:
        conn.autocommit = prev

## Errores comunes y diagnóstico rápido

- `OperationalError`: credenciales incorrectas, host/puerto erróneos, servicio caído o sin permisos de red.
- `ProgrammingError`: SQL inválido, tabla/columna inexistente.
- `IntegrityError`: violación de claves primarias/foráneas o restricciones.

Sugerencias:
- Verifica que el servidor PostgreSQL esté levantado y accesible.
- Revisa variables de entorno o credenciales.
- Asegúrate de que la BD `biblioteca` exista y tu usuario tenga permisos.

## Limpieza (opcional)

In [ ]:
# Drop de tablas de ejemplo (ejecuta solo si lo necesitas)
import psycopg2

def drop_tablas_demo(conn):
    try:
        with conn.cursor() as cur:
            cur.execute('DROP TABLE IF EXISTS libros_autocommit;')
        conn.commit()
        print('[OK] Tablas de demo eliminadas')
    except Exception as e:
        conn.rollback()
        print('[ERROR] Al eliminar tablas de demo:', e)